# Matemática Discreta A1 - Projeto Euler

Decidi aprender Julia para fazer o trabalho de Matemática Discreta. Essa linguagem é a queridinha do pessoal de otimização.

In [1]:
println("Hello world")

Hello world


Eu assisti um vídeo na internet chamado [The quickest introduction to Julia for pythonistas](https://www.youtube.com/watch?v=EJ7PLSrfj9M). Acaso esteja lendo este texto significa que eu não desisti no meio e fiz não fiz o trabalho em Python.

In [2]:
Fib(n) = if (n == 1 || n==0) 1 else Fib(n-1) + Fib(n-2) end

Fib (generic function with 1 method)

In [3]:
@time Φ = [if j==1 i else Fib(i) end for i in 1:15, j in 1:2]
Φ

  0.114585 seconds (209.06 k allocations: 10.213 MiB)


15×2 Array{Int64,2}:
  1    1
  2    2
  3    3
  4    5
  5    8
  6   13
  7   21
  8   34
  9   55
 10   89
 11  144
 12  233
 13  377
 14  610
 15  987

# [Problema 1](https://projecteuler.net/problem=1) (aquecimento)

If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23.

Find the sum of all the multiples of 3 or 5 below 1000.


In [4]:
Mult_Or(k,a,b) = if k % a == 0 || k % b == 0 k else 0 end

Mult_Or (generic function with 1 method)

In [5]:
sum_Helper(fun, max) = if max != 0 fun(max) + sum_Helper(fun,max-1) else 0 end
my_sum(max, a, b) = sum_Helper(((candidate) -> Mult_Or(candidate,a,b)), max-1)

my_sum (generic function with 1 method)

In [6]:
@time my_sum(1000,3,5)

  0.016012 seconds (11.07 k allocations: 581.936 KiB)


233168

In [7]:
@time my_sum(10,3,5)

  0.000004 seconds (4 allocations: 160 bytes)


23

In [8]:
@time sum([k for k = 1:999 if k % 3 == 0 || k % 5 == 0]) #mais simples, mas menos genérico, menos rápido e menos legal

  0.113356 seconds (66.94 k allocations: 3.373 MiB)


233168

In [9]:
my_sum1(m) = sum([k for k = 1:m if k % 3 == 0 || k % 5 == 0])

my_sum1 (generic function with 1 method)

In [10]:
@time my_sum1(1000)

  0.087142 seconds (81.84 k allocations: 4.128 MiB)


234168

In [11]:
@time my_sum1(1000)

  0.000030 seconds (15 allocations: 8.641 KiB)


234168

In [12]:
function my_Func2(max)
    resp = 0
    for k = 1:(max-1)
        
        if k % 3 == 0 || k% 5 == 0
            resp += k
        end
    end
    return resp
end

my_Func2 (generic function with 1 method)

In [13]:
@time my_Func2(1000) 

  0.017728 seconds (16.64 k allocations: 887.530 KiB)


233168

In [14]:
@time my_Func2(1000) #Aparentemente, o Julia tem um memoization rolando por debaixo dos panos

  0.000009 seconds (5 allocations: 176 bytes)


233168

# [Problema 2](https://projecteuler.net/problem=2)


Each new term in the Fibonacci sequence is generated by adding the previous two terms. By starting with $1$ and $2$, the first 10 terms will be:

$1, 2, 3, 5, 8, 13, 21, 34, 55, 89\ldots$

By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.


**Lema:** $F_n\ |\ 2  \Rightarrow n+1\ |\ 3$

Se $F_n$ é par, existem duas possíbilidades. Ou $F_{n-1}$ e $F_{n-2}$ são ambos pares, ou são ambos ímpares. Suponha por absurdo que ambos são pares, logo isso implicaria que $F_n$ é par para todo $n$. Mas existe $F_n$ ímpar, portanto não podem $F_{n-1}$ e $F_{n-2}$ ambos pares. No caso de serem ambos ímpares, logo nota-se um padrão. Considere a sequência $(\ldots,F_{n-3},F_{n-2},F_{n-1},F_{n},F_{n+1},F_{n+2},F_{n+3},\ldots)$. Adicionemos as informações que temos. $(\ldots,F_{n-3},F_{n-2}^i,F_{n-1}^i,F_{n}^p,F_{n+1},F_{n+2},F_{n+3},\ldots)$. Ora, conseguimos preencher o resto da sequência. $(\ldots,F_{n-3}^p,F_{n-2}^i,F_{n-1}^i,F_{n}^p,F_{n+1}^i,F_{n+2}^i,F_{n+3}^p,\ldots)$. Portanto $F_n$ é par a cada $3$ números de Fibonacci. Mas sabemos que $F_2$ é par, isso implica que $n+1|3$ para todo $n$, por conta da periodicidade das aparições.

In [15]:
function Euler2(max, this_Fib)
    flag = true
    index = 2
    sum = 0
    next_Fib = this_Fib(index)
    while flag
        sum += next_Fib
        index += 3
        next_Fib = this_Fib(index)
        flag = next_Fib <= max  
    end
    return sum
end

Euler2 (generic function with 1 method)

In [16]:
@time Euler2(4000000,Fib)

  0.116314 seconds (19.47 k allocations: 1002.483 KiB)


4613732

Estamos Fibonacci definida por recursão (escrevi no começo do ipynb sem saber que iria acabar usando apenas para verificar se eu sabia escrever uma função em Julia). Vamos tentar fazer melhor, com algoritmo $O(n)$?

Um professor de matemática discreta provou que:

$F_n = \dfrac{1}{\sqrt{5}}\left[\left(\frac{1+\sqrt{5}}{2}\right)^n-\left(\frac{1-\sqrt{5}}{2}\right)^n\right]$

In [17]:
Smart_Fib(n) = (((1+sqrt(5))/2)^(n+1) - ((1-sqrt(5))/2)^(n+1))/sqrt(5)

Smart_Fib (generic function with 1 method)

In [18]:
@time trunc(Int,Euler2(4000000,Smart_Fib))

  0.044375 seconds (57.60 k allocations: 3.066 MiB)


4613732

Como esperado, ao usar um algoritmo $O(n)$, conseguimos melhorar bastante o tempo de execução.

# [Problema 12](https://projecteuler.net/problem=12)


The sequence of triangle numbers is generated by adding the natural numbers. So the 7th triangle number would be 1 + 2 + 3 + 4 + 5 + 6 + 7 = 28. The first ten terms would be:

1, 3, 6, 10, 15, 21, 28, 36, 45, 55, ...

Let us list the factors of the first seven triangle numbers:

     1: 1
     3: 1,3
     6: 1,2,3,6
    10: 1,2,5,10
    15: 1,3,5,15
    21: 1,3,7,21
    28: 1,2,4,7,14,28

We can see that 28 is the first triangle number to have over five divisors.

What is the value of the first triangle number to have over five hundred divisors?


In [19]:
function is_Prime_helper(candidate, primes) #é divísivel por alguém na lista de primos
    for prime in primes
        if candidate % prime == 0
            return true
        end
    end
    return false
end

function is_Prime(candidate, primes)
    candidate = abs(candidate)
    if candidate == 1
        return (false, primes)
    end
    # se o candidato está na lista de primos
    if candidate in primes
        return (true, primes)
    end
    
    #se o candidato é divisível por alguém na lista de primos
    resp = is_Prime_helper(candidate, primes)
    if resp 
        return (false, primes)
    else
    
    #teste de primalidade clássico
    index = 5
    while index * index <= candidate
        if candidate % index == 0 || candidate % (index + 2) == 0
            return (false, primes)
        else
            index = index + 6
        end
    end
    primes = vcat(primes, [candidate])
    return (true,primes)
    end
end

is_Prime (generic function with 1 method)

In [20]:
function dict_of_divisors(div_Primos)
    nums = Dict()
    for elem in div_Primos
        if elem in keys(nums)
            nums[elem] = nums[elem] + 1
        else
            nums[elem] = 1
        end
    end
    return nums
end


function qtd_of_divisors(candidate,primes, iterador)
    if iterador == 1 iterador = iterador + 1 end
    while iterador < candidate
        bool, primes = is_Prime(iterador, primes)
        iterador = iterador + 1
    end
    resto = candidate
    div_Primos = []
    index = 1
    while resto != 1 || index <= size(primes)[1]
        while resto % primes[index] == 0
            x = primes[index]
            div_Primos = vcat(div_Primos,[primes[index]])
            resto = resto / primes[index]
        end
        index += 1
    end
    if size(div_Primos)[1] == 0
        primes = vcat(primes,[candidate])
        div_Primos = vcat(div_Primos,[candidate])
    end
    nums = dict_of_divisors(div_Primos)
    resp = 1
    for key in keys(nums)
        resp = resp * (nums[key] + 1)
    end
    return (resp,primes, iterador)
end


qtd_of_divisors (generic function with 1 method)

In [21]:
function exercicio12()
    triang = 3
    n = 2
    x, primes, iterador = qtd_of_divisors(triang, [2,3,5],1)
    while x < 500
        n = n + 1
        triang = triang + n
        x, primes, iterador = qtd_of_divisors(triang,primes,iterador)
    end
    return triang, x
end

exercicio12 (generic function with 1 method)

In [22]:
#@time exercicio12()

# Exercicio 20:



n! means n × (n − 1) × ... × 3 × 2 × 1

For example, 10! = 10 × 9 × ... × 3 × 2 × 1 = 3628800,
and the sum of the digits in the number 10! is 3 + 6 + 2 + 8 + 8 + 0 + 0 = 27.

Find the sum of the digits in the number 100!


Esse problema é equivalente a resolver o problema de encontrar a soma dos dígitos de $\frac{100!}{10^{10}}$

In [23]:
function list_of_primes_divisors(candidate, primes)
    bool, primes = is_Prime(candidate, primes)
    if bool
        return (primes, [candidate]) 
    end
    resto = candidate
    div_Primos = []
    index = 1
    while resto != 1 && index <= size(primes)[1]
        while resto % primes[index] == 0
            x = primes[index]
            div_Primos = vcat(div_Primos,[x])
            resto = resto / x
        end
        index += 1
    end
    if size(div_Primos)[1] == 0 
        div_Primos = vcat(div_Primos,[candidate])
    end
    return primes, div_Primos
end

list_of_primes_divisors (generic function with 1 method)

In [45]:
function primes_until(m)
    crivo = [true for i = 1:m]
    crivo[1] = false
    for num = 2:m
        if crivo[num] == true
            iter = num + num
            while iter <= m
                crivo[iter] = false
                iter = iter + num
            end
        end
    end
    return [iter for iter in 1:m if crivo[iter]]
end

function cem_Primos()
    return primes_until(100)
end

cem_Primos (generic function with 1 method)

In [46]:
primes_until(100)

25-element Array{Int64,1}:
  2
  3
  5
  7
 11
 13
 17
 19
 23
 29
 31
 37
 41
 43
 47
 53
 59
 61
 67
 71
 73
 79
 83
 89
 97

In [47]:
function exercicio20()
    primes = cem_Primos()
    factorial = []
    for n = 2:100
        primes, list_of = list_of_primes_divisors(n,primes)
        factorial = vcat(factorial, list_of)
    end
    dois, cinco = 0, 0
    index = 1
    while dois <= 19
        if factorial[index] == 2
            factorial[index] = 1
            dois = dois + 1
        end
        index = index + 1
    end
    while cinco <= 19
        if factorial[index] == 5
            factorial[index] = 1
            cinco = cinco + 1
        end
        index = index + 1
    end
    resp = BigInt(1)
    for elem in factorial
        resp = resp * BigInt(elem)
    end
    return sum(digits(resp))
end

exercicio20 (generic function with 1 method)

In [48]:
@time exercicio20()

  0.292257 seconds (392.19 k allocations: 19.728 MiB, 2.42% gc time)


648

# Problema 27

Euler discovered the remarkable quadratic formula:

$n^2+n+41$

It turns out that the formula will produce 40 primes for the consecutive integer values 0≤n≤39
. However, when n=40,402+40+41=40(40+1)+41 is divisible by 41, and certainly when n=41,412+41+41

is clearly divisible by 41.

The incredible formula n2−79n+1601
was discovered, which produces 80 primes for the consecutive values 0≤n≤79

. The product of the coefficients, −79 and 1601, is −126479.

Considering quadratics of the form:

    n2+an+b

, where |a|<1000 and |b|≤1000

where |n|
is the modulus/absolute value of n
e.g. |11|=11 and |−4|=4

Find the product of the coefficients, a
and b, for the quadratic expression that produces the maximum number of primes for consecutive values of n, starting with n=0.

In [49]:
function quadratic(a,b, primes)
    n = -1
    bool = true
    while bool
        n = n+1
        bool, primes = is_Prime(n^2 + a*n + b, primes)
        if !bool
        end
    end
    return n, primes
end

function problem27()
    primes = cem_Primos()
    champion = 0
    prod = 1
    for a=-999:999 
        for b=-1000:1000
            n, primes = quadratic(a,b,primes)
            champion = max(champion, n)
            if n == champion
                prod = a * b
            end
        end
    end
    return prod
end

problem27 (generic function with 1 method)

In [50]:
@time problem27()

  3.015984 seconds (56.13 k allocations: 6.848 MiB, 0.28% gc time)


-59231

# Problem 34



145 is a curious number, as 1! + 4! + 5! = 1 + 24 + 120 = 145.

Find the sum of all numbers which are equal to the sum of the factorial of their digits.

Note: as 1! = 1 and 2! = 2 are not sums they are not included.


Primeiramente, números curiosos devem ter um upper bound. Seja $n$ um número curioso com $m$ algarismos. Então $10^{m-1} \leq n \leq 9! m$. Isso deixa de ser verdade para $m\geq 8$. Portanto um upper bound é $10^7 = 10 000 000$.

In [51]:
function is_curious(n, fac)
    if n < 10
        return false
    end
    this_sum = 0
    algs = digits(n)
    index = 0
    this_size = size(algs)[1]
    while this_sum <= n && index < this_size
        index += 1
        this_sum += fac[algs[index]]
    end
    if this_sum == n
        return true
    else
        return false
    end
end

function problem34()
    fac = Dict()
    fac[0] = 1
    for num = 1:9
        fac[num] = fac[num - 1]*num
    end
    sum = 0
    for n = 1:10^7
        if is_curious(n, fac)
            sum += n
        end
    end
    return sum
end        

problem34 (generic function with 1 method)

In [52]:
@time problem34()

 11.233081 seconds (152.22 M allocations: 3.448 GiB, 1.72% gc time)


40730

# Problem 41


We shall say that an n-digit number is pandigital if it makes use of all the digits 1 to n exactly once. For example, 2143 is a 4-digit pandigital and is also prime.

What is the largest n-digit pandigital prime that exists?


Consideremos que $n \leq 9$.

In [44]:
using Combinatorics
function problem41()
    primes = cem_Primos()
    resp = 7
    for n = 2:9
        model = reduce(*,[string(i) for i = 1:n])
        elegibles = [parse(Int64,reduce(*,x)) for x in Combinatorics.permutations(model,n)]
        for elem in elegibles
            bool, primes = is_Prime(elem, primes)
            if bool
                resp = max(resp, elem)
            end
        end
    end
    return resp
end

problem41 (generic function with 1 method)

In [33]:
@time problem41()

  0.726957 seconds (8.50 M allocations: 356.260 MiB, 7.43% gc time)


7

# Problem 48



The series, $1^1 + 2^2 + 3^3 + ... + 10^{10} = 10405071317$.

Find the last ten digits of the series, $1^1 + 2^2 + 3^3 + ... + 1000^{1000}.$


Vamos usar congruência módulo $m$ e se lembrar que resolver este problema é a mesma coisa que resolver $x \equiv 1^1 + 2^2 + \cdots + 1000^{1000}\ (\mbox{mod}\ 10^{10})$.

In [53]:
function potencia_mod(n, m)
    resp = n
    for i = 2:n
        resp = (resp * n) % m
    end
    return resp
end

function problem48()
    sum_10 = 0
    for n = 1:1000
        parcela = potencia_mod(n, 10^(10))
        sum_10 = (sum_10 + parcela) % 10^(10)
    end
    return sum_10
end

problem48 (generic function with 1 method)

In [54]:
@time problem48()

  0.039581 seconds (56.87 k allocations: 3.094 MiB, 6.58% gc time)


9110846700

# Problem 55



If we take 47, reverse and add, 47 + 74 = 121, which is palindromic.

Not all numbers produce palindromes so quickly. For example,

349 + 943 = 1292,
1292 + 2921 = 4213
4213 + 3124 = 7337

That is, 349 took three iterations to arrive at a palindrome.

Although no one has proved it yet, it is thought that some numbers, like 196, never produce a palindrome. A number that never forms a palindrome through the reverse and add process is called a Lychrel number. Due to the theoretical nature of these numbers, and for the purpose of this problem, we shall assume that a number is Lychrel until proven otherwise. In addition you are given that for every number below ten-thousand, it will either (i) become a palindrome in less than fifty iterations, or, (ii) no one, with all the computing power that exists, has managed so far to map it to a palindrome. In fact, 10677 is the first number to be shown to require over fifty iterations before producing a palindrome: 4668731596684224866951378664 (53 iterations, 28-digits).

Surprisingly, there are palindromic numbers that are themselves Lychrel numbers; the first example is 4994.

How many Lychrel numbers are there below ten-thousand?

NOTE: Wording was modified slightly on 24 April 2007 to emphasise the theoretical nature of Lychrel numbers.


In [55]:
function is_Palindromic(n)
    nstring = string(n)
    len = length(nstring)
    for i = 1:Int(ceil(len/2))
        if nstring[i] != nstring[len-i+1]
            return false
        end
    end
    return true
end

function reverseInt(n)
    return parse(BigInt,reverse(string(n)))
end

function is_Lychrel(n)
    n = BigInt(n)
    counter = 1
    flag = true
    while counter <= 50
        n = n + reverseInt(n)
        if is_Palindromic(n)
            return false
        end
        counter += 1
    end
    return true
end

function problem55()
    counter = 0
    for n = 1:10000
        if is_Lychrel(n)
            counter += 1
        end
    end
    return counter
end

problem55 (generic function with 1 method)

In [56]:
@time problem55()

  0.192373 seconds (893.57 k allocations: 27.047 MiB, 4.98% gc time)


249

# Problem 63




The 5-digit number, $16807=7^5$, is also a fifth power. Similarly, the 9-digit number, $134217728=8^9$, is a ninth power.

How many n-digit positive integers exist which are also an nth power?


In [57]:
function is_n_power(num, n, primes)
    if n == 1
        return true
    end
    #anota quantos números primos cada número tem 
    list_of = list_of_primes_divisors(num, primes)[2]
    dict_divisors = dict_of_divisors(list_of)
    chaves = collect(keys(dict_divisors))
    first = dict_divisors[chaves[1]]
    if first % n != 0
        return false
    end
    for key in chaves[2:end]
        if dict_divisors[key] != first
            return false
        end
    end
    return true
end
#CRIADO MAS NÃO SERVIU

is_n_power (generic function with 1 method)

In [87]:
function problem63()
    powers = Dict()
    counter = 4
    for n = 1:9
        next = 1
        while next^n <= 10^n
            if length(string(next^n)) == n
                counter += 1
            end
            next = next + 1
        end
        counter += 1
    end
    return counter
end

problem63 (generic function with 1 method)

In [88]:
@time problem63()

  0.017491 seconds (20.77 k allocations: 1.129 MiB)


49